In [1]:
import pandas as pd
import numpy as np
import datetime
import os
# Add the new folders of Jian later

In [2]:
def recursive_file_gen(my_root_dir):
    for root, dirs, files in os.walk(my_root_dir):
        for file in files:
            yield os.path.join(root, file)
            
last_saturday=datetime.datetime.now().date()-datetime.timedelta(days=datetime.datetime.now().date().weekday()+2)

write_folder="/home/jian/celery/Append_to_celery_LiveRamp/monthly_update/output_"+str(last_saturday)+"/"
try:
    os.stat(write_folder)
except:
    os.mkdir(write_folder)

week_beginning=last_saturday-datetime.timedelta(days=14) # To be changed to 14

recent_2_weeks=[last_saturday-datetime.timedelta(days=x*7) for x in range(2)]

In [3]:
# Checking for the recent weeks that haven't moved to the folder
possible_recent_folders=["/home/jian/BigLots/MediaStorm_"+str(x)+"/" for x in recent_2_weeks]
recent_file_list=[]
for dirc in possible_recent_folders:
    list_recent=[x for x in list(recursive_file_gen(dirc)) if ("Master" in x) & (".txt" in x) ]
    recent_file_list=recent_file_list+list_recent
recent_file_df=pd.DataFrame({"path":recent_file_list,"date":[datetime.datetime.strptime(x.split("MasterWeekly")[1][:8],"%Y%m%d").date()-datetime.timedelta(days=3) for x in recent_file_list]},index=[x for x in range(len(recent_file_list))])


list_1_after_201806=[x for x in list(recursive_file_gen("/home/jian/BigLots/2019_by_weeks/")) if ("Master" in x) & (".txt" in x) ]
folder_date=[datetime.datetime.strptime(x.split("/")[len(x.split("/"))-2].split("_")[1],"%Y-%m-%d").date() for x in list_1_after_201806]
df_1_after_201806=pd.DataFrame({"date":folder_date,"path":list_1_after_201806},index=[x for x in range(len(list_1_after_201806))])
df_1_after_201806['date'].apply(lambda x: x.weekday()).unique()
df_1_after_201806=df_1_after_201806.sort_values("date").reset_index()
df_1_after_201806=df_1_after_201806[df_1_after_201806['date']>week_beginning]
del df_1_after_201806['index']

new_2_biweekly_files=df_1_after_201806.append(recent_file_df)

print(len(new_2_biweekly_files))
new_2_biweekly_files=new_2_biweekly_files['path'].unique().tolist()

2


In [4]:
new_2_biweekly_files

['/home/jian/BigLots/2019_by_weeks/MediaStorm_2019-05-11/MediaStormMasterWeekly20190514-121253-492.txt',
 '/home/jian/BigLots/MediaStorm_2019-05-18/MediaStormMasterWeekly20190521-111308-535.txt']

In [5]:
if len(new_2_biweekly_files)==2: # Changed to weekly
    df_all_new_master = pd.DataFrame()
    for file in new_2_biweekly_files:
        df = pd.read_csv(file,nrows = None,sep= '|',usecols = ['customer_id_hashed','email_address_hash','customer_zip_code'],
                         dtype =str)
        df_all_new_master = df_all_new_master.append(df,ignore_index = True)
    print(len(df_all_new_master.index))
    df_all_new_master = df_all_new_master.drop_duplicates('email_address_hash')
    print(len(df_all_new_master.index))

147779
147778


In [6]:
# Remove previous week ids

previous_id_files_lists=list(recursive_file_gen("/home/jian/Projects/Big_Lots/Live_Ramp/Quarterly_Update_2019Q1/crm_newscore_20190107/by_group_quadrant_24/"))

# previous_id_files_lists=[x for x in previous_id_files_lists if "Copy of 48 Stores_727.csv" not in x]
previous_id_files_lists=[x for x in previous_id_files_lists if ".csv" in x]
print(len(previous_id_files_lists))

72


In [7]:
import glob
uploaded_new_seg=list(recursive_file_gen("/home/jian/celery/Append_to_celery_LiveRamp/monthly_update/"))
uploaded_new_seg=[x for x in uploaded_new_seg if ".csv" in x]

previous_id_files_lists=previous_id_files_lists+uploaded_new_seg

In [8]:
previous_id_files_lists

['/home/jian/Projects/Big_Lots/Live_Ramp/Quarterly_Update_2019Q1/crm_newscore_20190107/by_group_quadrant_24/T_Quadrant I_P_L_2019Q1.csv',
 '/home/jian/Projects/Big_Lots/Live_Ramp/Quarterly_Update_2019Q1/crm_newscore_20190107/by_group_quadrant_24/T_Quadrant I_S_H_2019Q1.csv',
 '/home/jian/Projects/Big_Lots/Live_Ramp/Quarterly_Update_2019Q1/crm_newscore_20190107/by_group_quadrant_24/T_Quadrant I_S_L_2019Q1.csv',
 '/home/jian/Projects/Big_Lots/Live_Ramp/Quarterly_Update_2019Q1/crm_newscore_20190107/by_group_quadrant_24/T_Quadrant I_T_H_2019Q1.csv',
 '/home/jian/Projects/Big_Lots/Live_Ramp/Quarterly_Update_2019Q1/crm_newscore_20190107/by_group_quadrant_24/T_Quadrant I_T_M_2019Q1.csv',
 '/home/jian/Projects/Big_Lots/Live_Ramp/Quarterly_Update_2019Q1/crm_newscore_20190107/by_group_quadrant_24/C_Quadrant III_P_H_2019Q1.csv',
 '/home/jian/Projects/Big_Lots/Live_Ramp/Quarterly_Update_2019Q1/crm_newscore_20190107/by_group_quadrant_24/C_Quadrant III_P_M_2019Q1.csv',
 '/home/jian/Projects/Big_Lots

In [9]:
for file_uploaded in previous_id_files_lists:
    df = pd.read_csv(file_uploaded,usecols = ['email_address_hash'])
    previous_email_set=set(df['email_address_hash'].tolist())
    df_all_new_master = df_all_new_master[~df_all_new_master['email_address_hash'].isin(previous_email_set)]
    print(datetime.datetime.now(),len(df_all_new_master.index))

2019-05-22 11:04:37.671789 147777
2019-05-22 11:04:57.543829 147775
2019-05-22 11:04:58.309550 147775
2019-05-22 11:05:01.178064 147775
2019-05-22 11:05:04.744546 147774
2019-05-22 11:05:04.968198 147774
2019-05-22 11:05:05.081715 147774
2019-05-22 11:05:05.130359 147774
2019-05-22 11:05:05.169977 147774
2019-05-22 11:05:05.210068 147774
2019-05-22 11:05:05.290324 147774
2019-05-22 11:05:05.379662 147774
2019-05-22 11:05:05.419531 147774
2019-05-22 11:05:05.452858 147774
2019-05-22 11:05:05.491203 147774
2019-05-22 11:05:05.529556 147774
2019-05-22 11:05:05.608805 147774
2019-05-22 11:05:05.680245 147774
2019-05-22 11:05:05.712313 147774
2019-05-22 11:05:05.760982 147774
2019-05-22 11:05:05.793133 147774
2019-05-22 11:05:06.688536 147774
2019-05-22 11:05:08.345286 147774
2019-05-22 11:05:08.425201 147774
2019-05-22 11:05:08.491352 147774
2019-05-22 11:05:08.529365 147773
2019-05-22 11:05:45.247706 147772
2019-05-22 11:05:55.115391 147771
2019-05-22 11:06:00.492665 147768
2019-05-22 11:

In [10]:
df_all_new_master = df_all_new_master.drop_duplicates('email_address_hash')
df_all_new_master = df_all_new_master.drop_duplicates('customer_id_hashed')
print(len(df_all_new_master.index))

147749


In [11]:
zipmap = pd.read_csv('/home/jian/Projects/Big_Lots/New_TA/zips_in_new_ta/zip_with_ta_dma.csv',dtype = 'str')
zipmap['zipcodegroup'] = zipmap['revenue_flag']
zipmap = zipmap[['zip','zipcodegroup']].drop_duplicates('zip')
zipmap.columns = ['customer_zip_code','zipcodegroup']
df_all_new_master = pd.merge(df_all_new_master,zipmap,on ='customer_zip_code',how = 'left' )
print(df_all_new_master['zipcodegroup'].unique())
df_all_new_master['zipcodegroup'].fillna('T',inplace = True)
print(df_all_new_master['zipcodegroup'].unique())

last_saturday_str="_"+str(last_saturday).split("-")[1]+str(last_saturday).split("-")[2]+str(last_saturday).split("-")[0][2:]+"_"

for revenue_flag in ['P','S','T']:
    df = df_all_new_master[df_all_new_master['zipcodegroup']==revenue_flag]
    df = df[['customer_id_hashed','email_address_hash','customer_zip_code']]
    df['segment'] = 'NewReward' + last_saturday_str + revenue_flag
    print(revenue_flag,len(df.index))
    df.to_csv(write_folder+'NewReward_'+str(last_saturday)+'_'+revenue_flag+'.csv',index = False)
# To be uploaded to lr-big-lots uploads/big_lots_onboarding

['P' 'S' 'T' nan]
['P' 'S' 'T']
P 107295
S 26808
T 13646


In [12]:
df_all_new_master.groupby(['zipcodegroup'])['email_address_hash'].count().to_frame().reset_index()

,zipcodegroup,email_address_hash
0,P,107295
1,S,26808
2,T,13646


In [13]:
df.head(5)

,customer_id_hashed,email_address_hash,customer_zip_code,segment
9,5559159d170983cbf32fd347c92f633306d196d2aa3207...,cdb874e18e364f66a681be89ddc0aae5a83b7d58179b0a...,48422,NewReward_051819_T
15,090de3a71c1eac32cc0a0bd8ab47b6bfd097b3377a6922...,fa788c33be5cc646bf03b3df0439dfd8f943608e7350ad...,31092,NewReward_051819_T
25,772914a82a5a5626cb295713120c2022ff57c8ff12db2d...,35189b7e2edd3e4b9076a30f79dbbde643f8ffb91d3d49...,60631,NewReward_051819_T
30,1c89600789915e2234deb45d11c1984b1c6195b0d276ff...,20fda1c5b63a6e7255de799f25d14fd8cb262dce8a9b32...,30133,NewReward_051819_T
40,70f81880cc415c1e71148b3e897eacc1fb2fee23c77f26...,427ccc90779f33ad09b5cb0652d9c31ab213ed40af26c6...,74052,NewReward_051819_T


In [15]:
import paramiko
import glob

host = "files.liveramp.com"
port = 22
password = "Biglots2018!"
username = "lr-big-lots"

transport = paramiko.Transport((host, port))

transport.connect(username = username, password = password)
sftp = paramiko.SFTPClient.from_transport(transport)

# local_path defined above before saving the local txt
local_list=glob.glob(write_folder+"*.csv")
local_list=[x for x in local_list if 'NewReward_'+str(last_saturday)+'_' in x]

for local_path in local_list:
    remote_path="/uploads/big_lots_onboarding/"+os.path.basename(local_path)
    sftp.put(local_path,remote_path)
sftp.close()
